# Plotting.ipynb
Using the data generated by automate_anlaysis.py, we explore the relationship between SAR amplitude and IRI.


- created: 08.05.2020
- modified version of Pavement_quality_V2 colab notebook in drive 
- intended for the .pkl data output generated on or after 8/5
- intended for plotting only, now all processing and feature creation should be happening in automate_analysis.py

# Setup

In [ ]:
import math 
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
# Pandas display settings for previewing dataframes
pd.set_option('display.max_rows', 20)
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_columns', 40)

In [ ]:
def load_data_dict(dir):
    """
    in: path to directory with merged/cleaned .pkl datasets
    out: dictionary with all the merged SAR and IRI datasets as dataframes
    """
    datasets = {}

    for path in glob(dir + '*'):
        key = path.split('/')[-1][:-4]
        df = pd.read_pickle(path)
        datasets[key] = df
    
    return datasets

In [ ]:
DATA_DIR = '../data/road_level_merged/'
datasets = load_data_dict(DATA_DIR)

# IRI distribution

IRI density plots shouldn't be too surprising, this is mostly for comparison with the IRI density plot on p. 6 of the Meyer paper

In [ ]:
"""
IRI density function
"""
def IRI_density(df):
  # IRI for each road type
  sns.kdeplot(df.loc[(df['VDOT_Sys_I']=='Interstate'), 'NIRI_Avg'], color='black', bw=.1, Label='Interstate')
  sns.kdeplot(df.loc[(df['VDOT_Sys_I']=='Primary'), 'NIRI_Avg'], color='green', bw=.1, Label='Primary')
  sns.kdeplot(df.loc[(df['VDOT_Sys_I']=='Secondary'), 'NIRI_Avg'], color='blue',bw=.1, Label='Secondary')

  plt.xlabel('IRI')
  plt.ylabel('Probability Density')

In [ ]:
IRI_density(datasets['despeck_centerline_masked']) # arbitrary dataset, the IRI data is the same for all datasets

Analysis: Interstates have a much narrower IRI distribution than primaries and especially secondaries. This suggests that the latter two might also show more separability in SAR amplitude as well

# SAR amplitude distribution
- How is the SAR amplitude data spread?
- Mostly KDE plots here, also some boxplots

## By road type

The amplitude density distribution is also mostly for comparison with the Meyer paper, but we hope to see some separation between classes.

In [ ]:
"""
Amplitude distribution
"""

def amplitude_density(df, title):
  # Density plot for average amplitude
  
  plt.figure()
  sns.set(style='white')
  sns.kdeplot(df.loc[(df['VDOT_Sys_I']=='Interstate'), 'closest_mean'], color='black', Label='Interstate')
  sns.kdeplot(df.loc[(df['VDOT_Sys_I']=='Primary'), 'closest_mean'], color='green', Label='Primary')
  sns.kdeplot(df.loc[(df['VDOT_Sys_I']=='Secondary'), 'closest_mean'], color='blue', Label='Secondary')

  plt.xlim(0,2)
  plt.ylim(0,7)
  plt.xlabel('Amplitude')
  plt.ylabel('Probability Density')
  plt.title(title)

def all_amplitude_density(data_dict):
  for name, df in data_dict.items():
    amplitude_density(df, name)
    


In [ ]:
all_amplitude_density(datasets)

## By summary method

Here we are comparing the different methods for summarizing: mean, median and filtered_mean (mean of the pixels within the boxplot "min" and "max") 

Note: generating these plots can be slow

In [ ]:
def amplitude_kde_plots(data_dict):
  f, axes = plt.subplots(8, 1, figsize=(15, 40))
  ax = 0

  # each dataset gets a new plot
  for name, df in data_dict.items():
    # each summary method gets a new kde line
    for method in ['mean', 'filtered_mean', 'median']:
      amp = df.filter(regex='\d_'+method).to_numpy().flatten()
      sns.distplot(amp,
                  hist=False,
                  kde_kws={'bw' : 0.1, 
                          'gridsize' : 1000, 
                          'label' : '{} {} ({})'.format(name, method, len(amp))}, 
                  fit=None, 
                  fit_kws={'linestyle' : ':',
                          'label' : 'Fit for {}'.format(name)},
                  ax=axes[ax])
      
      axes[ax].set(xlim=(0, 2), ylim=(0, 3.5))
      axes[ax].grid(True)

    ax += 1

In [ ]:
amplitude_kde_plots(datasets)

## By pavement quality category

Quality ratings as defined by Virginia DOT:

#### Interstate and Primary
Ride quality | IRI rating (in/mile) |
--- | ---
Excellent |  < 60
Good | 60 - 99
Fair | 100 - 139
Poor | 140 - 199
Very Poor | >= 200

#### Secondary
Ride quality | IRI rating (in/mile) |
--- | ---
Excellent |  < 95
Good | 95 - 169
Fair | 170 - 219
Poor | 220 - 279
Very Poor | >= 280



In [ ]:
# Quality ratings as defined by VDOT
quality = {'Interstate' : {'Excellent' : (0, 60),
                           'Good' : (60, 100),
                           'Fair' : (100, 140),
                           'Poor' : (140, 200), 
                           'Very poor': (200, np.inf)},
           'Primary' : {'Excellent' : (0, 60),
                        'Good' : (60, 100),
                        'Fair' : (100, 140),
                        'Poor' : (140, 200), 
                        'Very poor': (200, np.inf)},
           'Secondary' : {'Excellent' : (0, 95),
                           'Good' : (95, 170),
                           'Fair' : (170, 220),
                           'Poor' : (220, 280), 
                           'Very poor': (280, np.inf)}
           }

def quality_density(df, quality, stat):
  """
  Plotting amplitude KDE and boxplots by quality
  Parameters:
    df: dataframe 
    quality: dictionary of quality categories (above)
    stat: statistic column to visualize, e.g. 'avg_mean'
  """
  bw = 0.01  # The bandwidth of the Gaussian used. Making this small shows where the actual measurements are

  f, axes = plt.subplots(2, 3, figsize=(15, 15))
  ax = 0

  # For each type of road
  for (road_type, ranges) in quality.items():

    # For each riding condition
    for (ride, values) in ranges.items():

      filtered = df.loc[(df['NIRI_Avg'] >= values[0]) & (df['NIRI_Avg'] < values[1]) & (df['VDOT_Sys_I'] == road_type), stat]
      label = '{} {}'.format(ride, values)
      sns.kdeplot(filtered, bw=bw, Label=label, ax=axes[0, ax]).set_title(road_type)      

    # Box plot for each road type
    filtered = df[df['VDOT_Sys_I'] == road_type]
    sns.set(style='whitegrid', palette='colorblind')
    sns.boxplot(x='quality', y=stat, hue='VDOT_Sys_I', data=filtered, ax=axes[1, ax], notch=True, order=['Excellent', 'Good', 'Fair', 'Poor', 'Very poor'])
    #axes[1, ax].set(ylim=(0, 1.25))
    ax += 1 



In [ ]:
quality_density(datasets['despeck_buffered_masked'], quality, 'closest_mean')

# Meyer (2016) Calibration Roads

Here, we used QGIS to manually select the roads that it looked like Meyer used in the 2016 report

In [ ]:
# select the roads Meyer used based Route_Name and more
def select_meyer_roads(df):

    df = df.loc[(df['Year'] == '2012') | (df['Year'] == '2014')]

    meyerinterstates = df.loc[((df['Route_Name'] == 'R-VA   IS00081SB') & (df['Start_GPS_'] < 38.108)) | \
                              ((df['Route_Name'] == 'R-VA   IS00081NB') & (df['Start_GPS_'] < 38.108))]

    meyersecondary = df.loc[((df['Route_Name'] == 'R-VA007SC00876NB') & (df['Start_GPS_'] < 38.130) & (df['Year'] == '2012')) | \
                            ((df['Route_Name'] == 'R-VA007SC00662EB') & (df['Start_GPS1'] < -79.174) & (df['Year'] == '2012')) | \
                            ((df['Route_Name'] == 'R-VA007SC00919NB') & (df['Year'] == '2012')) | \
                            ((df['Route_Name'] == 'R-VA007SC00703EB') & (df['Year'] == '2012')) | \
                            ((df['Route_Name'] == 'R-VA007SC00604NB') & (df['Start_GPS_'] < 38.010) & (df['Year'] == '2012')) | \
                            ((df['Route_Name'] == 'R-VA007SC00670NB') & (df['Year'] == '2012')) | \
                            ((df['Route_Name'] == 'R-VA007SC00720EB') & (df['Start_GPS_'] < 38.214) & (df['Year'] == '2014')) | \
                            ((df['Route_Name'] == 'R-VA007SC00723NB') & (df['Year'] == '2014')) | \
                            ((df['Route_Name'] == 'R-VA007SC00612EB') & (df['Start_GPS_'] < 38.195) & (df['Year'] == '2014')) | \
                            ((df['Route_Name'] == 'R-VA007SC00792NB') & (df['Year'] == '2014')) | \
                            ((df['Route_Name'] == 'R-VA007SC00654EB') & (df['Year'] == '2014')) | \
                            ((df['Route_Name'] == 'R-VA007SC00701EB') & (df['Year'] == '2014')) | \
                            ((df['Route_Name'] == 'R-VA007SC00708NB') & (df['Year'] == '2014'))]

    meyerroads = pd.concat([meyerinterstates, meyersecondary])

    return meyerroads


In [ ]:
# select Meyer's subset of the data
df = select_meyer_roads(datasets['despeck_buffered_masked'])

# amplitude KDE
amplitude_density(df, title='')
plt.show()

# IRI KDE
IRI_density(df)
plt.show()

# amplitude vs IRI
sns.scatterplot(x='closest_mean', y='NIRI_Avg', hue='VDOT_Sys_I', legend=False, data=df)
plt.xlim(0, 2)
plt.ylim(0, 400)

Analysis: the plots here suggest that the subset of the data that Meyer used doesn't behave notably differently from the trends (or lack there of) if we look at all road segments.

# Simple SAR vs IRI scatterplots

Let's plot IRI vs various SAR amplitude metrics
- All these metrics are different ways of aggregating the pixel level SAR amplitude data to the road level, so we can compare the road level IRI data points to the SAR data 
- All of these metrics are calculated using only the SAR images that were acquired at dates close to the IRI acquisition date for a given road segment. Our thinking is that this is important because an underlying idea/assumption is that road quality changes over time

In [ ]:
# for each road type
#   for each SAR amplitude feature 
#     scatterplot SAR amplitude feature vs NIRI

df = datasets['despeck_buffered_masked']
sar_features = ['closest_mean', 'closest_std', 'closest_median', 'closest_iqr', \
                'pf_slope', 'ts_slope', 'gauss_closest_mean', 'gauss_closest_std'] 

for x in sar_features:  
  sns.set(style='whitegrid', palette='colorblind')
  g = sns.FacetGrid(df, col='VDOT_Sys_I', col_wrap=3, hue='VDOT_Sys_I', height=5)
  g.map(sns.scatterplot, x, 'NIRI_Avg', alpha=0.7, s=25)
  g.add_legend()

Analysis: this is where we would hope to see a clearer trend, especially between mean SAR and IRI where Meyer reported a trend. 

# Acquisition Date Scatterplots 

These plots divide subplots based on the closest SAR acquisition date instead of plotting data for each image. 

There's pretty siginificant variation in the number of points that appear in each plot, due to the differences in IRI acquisition date and SAR acquisition date. 

In [ ]:
def sar_date_scatters(df, column):
  """
  this divides subplots by the closest SAR acquisition date instead of the image
  """

  # sort so that interstate vals are visible!
  df = df.sort_values(by='VDOT_Sys_I', ascending=False)

  sns.set(style='whitegrid', palette='colorblind')
  g = sns.FacetGrid(df, col='closest_date', col_wrap=4, hue='VDOT_Sys_I')
  g.map(plt.scatter, column, 'NIRI_Avg', alpha=0.7, s=5)
  g.add_legend()

In [ ]:
# MEAN 
sar_date_scatters(datasets['despeck_buffered_masked'], column='closest_mean')

In [ ]:
# STD DEV 
sar_date_scatters(datasets['despeck_buffered_masked'], column='closest_std')

# Amplitude vs all pavement quality metrics (alligator cracking, CCI, IRI...) 

In [ ]:
# for each road type
#   for each of the pavement quality measurements
#     scatterplot mean SAR amplitude vs measurement
df = datasets['despeck_buffered_masked']
pavement_quality_measurements = ['CCI', 'LDR', 'NDR', 'LM_Valid_I', 'LM_Invalid', 'NIRI_Avg',
       'IRI_Left_W', 'IRI_Right_', 'LM_EBcelle', 'LM_Good_IR', 'LM_Fair_IR',
       'LM_Poor_IR', 'LM_Very_Po', 'Transverse', 'Transver_1', 'Longitudin',
       'Longitud_1', 'Longitud_2', 'Longitud_3', 'Reflective', 'Reflecti_1',
       'Reflecti_2', 'Reflecti_3', 'Reflecti_4', 'Reflecti_5', 'Alligator_',
       'Alligator1', 'Alligato_1', 'Patching_A', 'Patching_1', 'Number_of_',
       'Delaminati', 'Bleeding_S', 'Bleeding_1', 'Distress_L',
       'No_Distres']
for y in pavement_quality_measurements:  
  sns.set(style='whitegrid', palette='colorblind')
  g = sns.FacetGrid(df, col='VDOT_Sys_I', col_wrap=3, hue='VDOT_Sys_I', height=5)
  g.map(sns.scatterplot, 'closest_mean', y, alpha=0.7, s=25)
  g.add_legend()

# Amplitude boxplots per image

Here, we compare the distributions of average amplitude values of road segments between the 67 images in the stack. Boxplots are with and without outliers.


In [ ]:
def amp_boxplot_outliers(df, year):

  # only use columns with amplitude values
  amp_vals = df.filter(regex=r'filtered_mean$')
  amp_vals = amp_vals.reindex(sorted(amp_vals.columns), axis=1)

  # filter for only the given year
  amp_vals = amp_vals.filter(like=year)
  
  #PLOT!
  sns.set(style="darkgrid", font_scale=1, color_codes=True, palette="deep")
  sns.boxplot(data=amp_vals, fliersize=3, orient="h")
  plt.xlabel("SAR amplitude values")
  plt.ylabel("Date")
  plt.title("SAR amplitude values in " + year)
  plt.figure()
  

def amp_boxplot_no_outliers(df, year):
  
  # only use columns with amplitude values
  amp_vals = df.filter(regex=r'filtered_mean$')
  amp_vals = amp_vals.reindex(sorted(amp_vals.columns), axis=1)

  
  # filter for only the given year
  amp_vals = amp_vals.filter(like=year)
  
  #PLOT!
  sns.set(style="darkgrid", font_scale=1, color_codes=True, palette="deep")
  sns.boxplot(data=amp_vals, fliersize=3, orient="h", showfliers=False)
  plt.xlabel("SAR amplitude values")
  plt.ylabel("Date")
  plt.title("SAR amplitude values in " + year)
  plt.figure()

In [ ]:
data = datasets['despeck_buffered_masked']

amp_boxplot_outliers(data, '2011')
amp_boxplot_outliers(data, '2012')
amp_boxplot_outliers(data, '2013')
amp_boxplot_outliers(data, '2014')
amp_boxplot_no_outliers(data, '2011')
amp_boxplot_no_outliers(data, '2012')
amp_boxplot_no_outliers(data, '2013')
amp_boxplot_no_outliers(data, '2014')

# Multiple Feature Scatterplots
- looking to separate IRI classes based on multiple SAR features (rather than just one feature as in Simple SAR vs IRI scatterplots):
 - std and mean
 - iqr and median
 - mean and rate of change in amplitude

In [ ]:
sns.set(style='whitegrid', palette='colorblind')

def quality_class_scatterplots(x, y, df):
  g = sns.FacetGrid(df, col='VDOT_Sys_I', hue='quality', height=5, xlim=(0, 2.5))
  g.map(sns.scatterplot, x, y, alpha=0.7, s=20)
  g.add_legend()

# despeck
df = datasets['despeck_centerline_masked'] 
quality_class_scatterplots('closest_mean', 'closest_std', df)
quality_class_scatterplots('closest_median', 'closest_iqr', df)
quality_class_scatterplots('closest_mean', 'pf_slope', df)

# raw
df = datasets['raw_centerline_masked']

quality_class_scatterplots('closest_mean', 'closest_std', df)
plt.ylim(0, 1.6)

quality_class_scatterplots('closest_median', 'closest_iqr', df)
plt.ylim(0, 1.6)

quality_class_scatterplots('closest_mean', 'pf_slope', df)